In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import old.tools, old.RNTK_avg
import jax
import symjax
import symjax.tensor as T
import copy
from RNTK_NEW import RNTK

In [3]:
dataset = "trains"
# dataset = "bank"

In [4]:
dic = {k:v for k,v in map(lambda x : x.split(), open("data" + "/" + dataset + "/" + dataset + ".txt", "r").readlines())}

In [35]:
# N = int(dic["n_patrons1="])
# length = int(dic["n_entradas="])
N = 1000
length = 100

In [36]:
DATA = T.Placeholder((N, length), 'float32', name = "X")
x = DATA[:,0]
X = x*x[:, None]
n = X.shape[0]

In [37]:
from RNTK_onediag import RNTK as RNTKOD
rntkod = RNTKOD(dic, 2, 2, X, n) #could be flipped

In [38]:
lin_ema = rntkod.create_func_for_diag(fbool = True)

In [39]:
final_ema = rntkod.no_bc_arrays_to_diag(lin_ema)

In [40]:
final_ema

(Op(name=concatenate_60, fn=concatenate, shape=(3, 3, 1000, 1000), dtype=float32, scope=/default),
 Op(name=concatenate_61, fn=concatenate, shape=(3, 3, 1000, 1000), dtype=float32, scope=/default))

In [41]:
rntk_func = symjax.function(DATA, outputs=final_ema) #works fine for shizzle

In [44]:
fake_data = np.zeros((N,length))

In [45]:
gp_kernel = rntk_func(fake_data)[0]
rntk_kernel = rntk_func(fake_data)[1]

In [46]:
gp_kernel.shape

(3, 3, 1000, 1000)